In [ ]:
!pip install pygsheets

In [1]:
import pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pygsheets
import pandas as pd

In [2]:
def google_sheets(file, data):
    """
    Fonction pour l'export vers googlesheets
    """
    #authentification
    gc = pygsheets.authorize(service_file= '/home/jovyan/auth/biofitec-445d3a2a47d2_sheets.json')
    
    # on ouvre de fichier excel
    sh = gc.open(file)
    
    # on selectionne la première feuille
    wks = sh[0]
    
    # on ecrit 
    wks.set_dataframe(data.toPandas(), (1,1))
    

### I - Connexion

In [3]:
# connexion du driver
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook2").\
        master("spark://172.17.0.1:8077").\
        config("spark.executor.memory", "1g").\
        config("spark.mongodb.input.uri","mongodb://172.17.0.1:28117,172.17.0.1:28118,172.17.0.1:28119/BIO.departements?replicaSet=rs0").\
        config("spark.mongodb.output.uri","mongodb://172.17.0.1:28117,172.17.0.1:28118,172.17.0.1:28119/BIO.departements?replicaSet=rs0").\
        config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
        getOrCreate()

### II - Récupération des données

In [4]:
df = spark.read.format("mongo").load()

### III - Magasins Bio

In [5]:
df.createOrReplaceTempView("mags")

In [6]:
magsDF = spark.sql("select departement, size(communes) as communes, magasins.* from mags")

In [7]:
magsDF = magsDF.fillna(0)

In [8]:
magsDF.show()

+--------------------+--------+----------+-------+--------+----------+-------------+-------------------+---------+-------+
|         departement|communes|BIO C' BON|BIOCOOP|BIOMONDE|L'EAU VIVE|LA VIE CLAIRE|LE GRAND PANIER BIO|NATURALIA|NATUREO|
+--------------------+--------+----------+-------+--------+----------+-------------+-------------------+---------+-------+
|                 AIN|     410|         0|      3|       0|         0|            4|                  1|        0|      0|
|               AISNE|     805|         0|      2|       1|         0|            3|                  0|        0|      0|
|              ALLIER|     318|         0|      2|       1|         0|            5|                  0|        0|      0|
|ALPES-DE-HAUTE-PR...|     199|         0|      5|       0|         0|            3|                  0|        1|      0|
|        HAUTES-ALPES|     168|         0|      8|       1|         1|            1|                  0|        0|      0|
|     ALPES-MARI

In [9]:
google_sheets(file= "mags_bio_departs", data= magsDF)